In [2]:
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.119:7077") \
        .appName("TianruZ_example_shared_state")\
        .config("spark.executor.cores",2)\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled", True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()


# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

In [3]:
# Immutable data for all workers
broadcast_variable = spark_context.broadcast({'apples': 100, 'oranges': 42, 'pears': 1})

In [4]:
rdd = spark_context.parallelize(['apples', 'pears'])

# Convert fruit to amounts of fruit.
# 'value' is captured in the closure, but its only a handle - the dictionary is serialized via a different route
result = rdd.map(lambda x: broadcast_variable.value[x])\
            .collect()

print(result)

[100, 1]


In [5]:
# ACCUMULATOR 

accumulator_1 = spark_context.accumulator(0)

rdd2 = spark_context.parallelize([1,2,3,4,5,6],2)



In [6]:
result = rdd2.foreach(lambda x: accumulator_1.add(1))

print(accumulator_1.value)

6


In [7]:
# release the cores for another application!
spark_context.stop()